In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools #own file with helper

In [ ]:
print cvxpy.installed_solvers()

In [ ]:
###################################################
# Boris Diss Hydraulisches Positioniersystem S.61 #
###################################################
A = np.matrix([[0,      1,        0],
               [0, -0.126,     83.5],
               [0,  -6.89, -0.00707]])
 
b = np.matrix([[0],[0],[-55.51]])
c = np.matrix([1, 0, 0])

print "Poles:\n", np.matrix(con.matlab.pole(con.matlab.ss(A, b, c, 0))).T

u_max = 1
n = len(b)

X0 = [np.matrix([-13.38, -10.7, -2.58]).T,
      np.matrix([-13.38, -10.7,  2.58]).T,
      np.matrix([-13.38,  10.7, -2.58]).T,
      np.matrix([-13.38,  10.7,  2.58]).T,
      np.matrix([ 13.38, -10.7, -2.58]).T,
      np.matrix([ 13.38, -10.7,  2.58]).T,
      np.matrix([ 13.38,  10.7, -2.58]).T,
      np.matrix([ 13.38,  10.7,  2.58]).T]

x_max = [13.38, 10.7, 2.58]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c
p_min = 0.2
mu_star = 1.5
zeta = 2.5
zeta_star = 1.5

k0 = 1e-3*np.matrix([-24.23, -0.2122, -10.5]).T
k1 = 1e-3*np.matrix([-40.13, -0.1396, -15.10]).T
k0_star = 1e-3*np.matrix([-11.96, -0.1111, -5.18]).T
k1_star = 1e-3*np.matrix([-83.35, -0.0477, -33.86]).T

##### Entwurf parameter #####
beta=2 # beta >=1

In [ ]:
def someparameters():
    k0 = np.matrix([[ 0.00702601],
     [-0.79417547],
     [-0.01583519]])

    k1 = np.matrix([[ 0.00044966],
     [-0.02280205],
     [-0.00489574]])

    k0_star = np.matrix([[ 0.00151984],
     [-0.2060622 ],
     [-0.00826113]])

    k1_star = np.matrix([[ 0.00045032],
     [-0.02262498],
     [-0.00470835]])


In [ ]:
### Design Sättigungsregler mittels konvexer Hülle (A.3)

# Variables (Convex)
#  Name in A.3 | Name in Program
#    Q  = P^⁻1 |   = Q
#    z  = Ql   |   = z0
#    z* = Ql*  |   = z1

# Variables (Quasi convex)
#    gamma     |   = g

# Parameter
#    mu        |   = m (Designparameter)
#    X0        |   = X0 = [x0,...]
#    A         |   = A
#    b         |   = b

# Initialize
n = len(b) # get dim of system
m = 1

# Define Variables
#Q  = cvxpy.Semidef(n) #symmetric and positive semidefinite
Q  = cvxpy.Variable(n, n) # Semidef could go as an additional constraint as well, thereby no need fo Q to be symmetric
const_sdQ = (Q >> 0) # semidef but not (necessarily) symmetric

z0 = cvxpy.Variable(n)
z1 = cvxpy.Variable(n)

# Bisection parameter
g = cvxpy.Parameter(sign='positive')
#g.value = g_val # TODO set by bisection loop (function?)

# Define Constraints
# (A.10)
const_A10 = [cvxpy.bmat([[Q,       X0[i]],
                         [X0[i].T, 1    ]]) >> 0
                            for i in range(0, len(X0))]

# (A.11)
const_A11 = cvxpy.bmat([[Q,  z0],
                        [z0.T, 1 ]]) >> 0

# (A.12)
const_A12 = cvxpy.bmat([[Q,    z1  ],
                        [z1.T,   m**2]]) >> 0

# (A.13)
const_A13 = Q*A + A.T*Q - b*z0.T - z0*b.T << 0 # This constraint is strict definit

# (A.14)
const_A14 = Q*A + A.T*Q - b*z1.T - z1*b.T << -2*g*Q # This constraint is strict definit

constraints = [const_sdQ]
constraints.extend(const_A10) ##!! Beware of the "extend" if input is array
constraints.append(const_A12)
constraints.append(const_A13)
constraints.append(const_A14)

# Define Objective (Generalized eigenvalue problem?)
#obj = cvxpy.Maximize(g)

# Feasibility for bisection:
obj = cvxpy.Minimize(0)

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)

In [ ]:
# Test solution with bisection parameter g=1
g.value = 1
prob.solve(solver=cvxpy.SCS, verbose=False, max_iters=2500)  # Returns the optimal value.
#prob.solve()  # Returns the optimal value.

prob.status

In [ ]:
# Perform bisection on g in {[0,2]}
[[o_Q, o_z0, o_z1], o_g] = optim_tools.bisect_max(
    0, 2, prob, g, [Q, z0, z1], solver=cvxpy.SCS, verbose=1)

In [ ]:
# A.4
# Transformationen

In [ ]:
import cvxpy

R0 = cvxpy.Variable(n,n)
R1 = cvxpy.Variable(n,n)

G = cvxpy.Variable(n,n) #skew
G_star = cvxpy.Variable(n, n) # skew

# D = cvxpy.Symmetric(n)
# D_star = cvxpy.Symmetric(n)
D = cvxpy.Variable(n, n) #sym
D_star = cvxpy.Variable(n, n) #sym

constraints = [D                 == D.T, # sym
               D_star            == D_star.T, # sym
               G + G.T           == 0, # skew
               G_star + G_star.T == 0] # skew


# Parameter
alpha = 1-p_min
beta = 1+p_min
    
A0 = A-b*k0.T
A1 = A-b*k1.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

A0_star = A-b*k0_star.T
A1_star = A-b*k1_star.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

S0_star = A1_star.T*R1+R1*A1_star
S1_star = A0_star.T*R1+R1*A0_star + A1_star.T*R0+R0*A1_star
S2_star = A0_star.T*R0+R0*A0_star

# Create two constraints.
const4_39ab = [R0 >> 0,
               R0 + R1 >> 0]

const4_39c = cvxpy.bmat([[-D , G],
                         [G.T, D]]) - \
             cvxpy.bmat([[S0 + 0.5*beta*S1 + 0.25*beta**2*S2, 0.25*alpha*(S1 + beta*S2)],
                         [0.25*alpha*(S1 + beta*S2),          0.25*alpha**2*S2]]) >> 0

const4_39d = [cvxpy.bmat([[1, X0[i].T],
                          [X0[i], R0+R1]]) >> 0
                              for i in range(0, len(X0))]

const4_39e = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             1/p_min * cvxpy.bmat([[0,  k1.T],
                                  [k1, R1]]) >> 0

const4_39f = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             cvxpy.bmat([[0,  k1.T],
                         [k1, R1]]) >> 0

const4_39g = cvxpy.bmat([[-D_star , G_star],
                         [G_star.T, D_star]]) - \
             cvxpy.bmat([[S0_star + 0.5*beta*S1_star + 0.25*beta**2*S2_star, 0.25*alpha*(S1_star + beta*S2_star)],
                         [0.25*alpha*(S1_star + beta*S2_star),          0.25*alpha**2*S2_star]]) >> 0

constraints.extend(const4_39ab) ##!! Beware of the "extend" if input is array
constraints.append(const4_39c)
constraints.extend(const4_39d) ##!! Beware of the "extend" if input is array
constraints.append(const4_39e)
constraints.append(const4_39f)
constraints.append(const4_39g)

# Form objective.
obj = cvxpy.Minimize(cvxpy.trace(R0+1/p_min*R1))

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS, verbose=True, max_iters=2500)  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
print "R0", R0.value
LA.cholesky(R0.value)
print "R1", R1.value
LA.cholesky(R1.value + R0.value)
print "ok"

In [ ]:
def k_explizit(roots_p1, roots_pmin, pmin, a):
    raise ValueError("not working correctly")
    n = len(roots_p1)
    
    k0 = np.zeros((n,))
    k1 = np.zeros((n,))

    a_tilde_p1 = np.poly(roots_p1)[1:]
    a_tilde_pmin = np.poly(roots_pmin)[1:]

    for i in range(0, n):
        k1[i] = (a_tilde_p1[i] - a_tilde_pmin[i]) / (1.0-1.0/pmin)
        k0[i] = a_tilde_p1[i] - a.item(i) - k1[i]
    return np.matrix(k0).T, np.matrix(k1).T

k_explizit([1,2,4], [2,4,8], 0.1, np.matrix([0, 1, 2]))

In [ ]:
import control
import matplotlib.pyplot as plt

# uboot
A = np.matrix([[0., 1., 0.],
              [0., 0., 1.],
              [0., 0., -0.005]])
a = -A[-1][:].T #!!!!

b = np.matrix([[0], [0], [1.]])

d = 0
c = np.matrix([[1], [0], [0]])

sys = control.ss(A, b, c.T, d)
#mag, phase, omega = bode(sys1)
#arr1, arr2 = control.step(sys)
#plt.plot(arr2, arr1)

#poles, zeros = control.matlab.pzmap(sys, True)
plt.axis([-5,.1,-3,3])
#plt.show

#k0, k1 = k_explizit([-1,0,0], [-1,0,0], 0.1, a)
pmin = 0.1

r0 = control.place(A, b, [-1, -1+1j, -1-1j]) #k(p=1)
r1 = control.place(A, b, [-3, -3+2j, -3-2j]) #k(p=pmin)

# This seems to work as expected
k1 = 1.0/(1.0-1.0/pmin) * (r0 - r1)
k0 = r0 - k1

for p in np.arange(1, 0.09, -0.01):
    #sys_cl = control.ss(A-b*(r0), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #sys_cl = control.ss(A-b*(r1), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print p
    sys_cl = control.ss(A-b*(k0+1.0/p*k1), b, c.T, d)
    poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print poles2
#plt.show